In [51]:
import os
import glob
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pprint
import pyspark
import pyspark.sql.functions as F

from pyspark.sql.functions import col, to_date
from pyspark.sql.types import StringType, IntegerType, FloatType, DateType

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score, roc_auc_score
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split




In [52]:
# Build a .py script that takes a snapshot date, loads a model artefact and make an inference and save to datamart

## set up pyspark session

In [53]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

## set up config

In [54]:
snapshot_date_str = "2024-06-01"
model_name = "credit_model_2024_06_01.pkl"


In [55]:
config = {}
config["snapshot_date_str"] = snapshot_date_str
config["snapshot_date"] = datetime.strptime(config["snapshot_date_str"], "%Y-%m-%d")
config["model_name"] = model_name
config["model_bank_directory"] = "/app/model_bank/"
config["model_artefact_filepath"] = config["model_bank_directory"] + config["model_name"]

pprint.pprint(config)

{'model_artefact_filepath': '/app/model_bank/credit_model_2024_06_01.pkl',
 'model_bank_directory': '/app/model_bank/',
 'model_name': 'credit_model_2024_06_01.pkl',
 'snapshot_date': datetime.datetime(2024, 6, 1, 0, 0),
 'snapshot_date_str': '2024-06-01'}


## load model artefact from model bank

In [56]:
# Load the model from the pickle file
with open(config["model_artefact_filepath"], 'rb') as file:
    model_artefact = pickle.load(file)

print("Model loaded successfully! " + config["model_artefact_filepath"])

Model loaded successfully! /app/model_bank/credit_model_2024_06_01.pkl


## load feature store

In [57]:
# connect to feature store
folder_path = "/app/datamart/gold/feature_store/"
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
feature_sdf = spark.read.option("header", "true").parquet(*files_list)

# Ensure snapshot_date is in proper DateType
feature_sdf = feature_sdf.withColumn("snapshot_date", to_date(col("snapshot_date"), "yyyy-MM-dd"))

print("row_count:", feature_sdf.count())
feature_sdf.show()

# extract feature store
feature_sdf = feature_sdf.filter((col("snapshot_date") == config["snapshot_date"]))
print("extracted features_sdf", feature_sdf.count(), config["snapshot_date"])

feature_pdf = feature_sdf.toPandas()
feature_pdf

row_count: 104288
+-----------+-------------+---------------------+---+----------+-------------------+----------------+-----------------------+-------------+-----------------+---------------+-------------------+-----------------------+----------------------+-------------------+-----------------------+-----------------------------+--------------------------+---------------------------------+-----------------------+-----------------------+---------------------+------------+-------------+------------+----------------+---------------+--------------------+---------------------+--------------------+----------------+-----------------+--------+-------+----------+------------------------+---------------------+--------------------+--------------------+---------------------+--------------------+---------------------+--------------------+----------------------+---------------------+
|Customer_ID|snapshot_date|feature_snapshot_date|Age|Occupation|Delay_from_due_date|Outstanding_Debt|Amount_invested

extracted features_sdf 5418 2024-06-01 00:00:00


,Customer_ID,snapshot_date,feature_snapshot_date,Age,Occupation,Delay_from_due_date,Outstanding_Debt,Amount_invested_monthly,Interest_Rate,Num_Bank_Accounts,...,clickstream_total_events,clickstream_fe_5_mean,clickstream_fe_5_sum,clickstream_fe_5_std,clickstream_fe_9_mean,clickstream_fe_9_min,clickstream_fe_4_mean,clickstream_fe_4_min,clickstream_fe_10_mean,clickstream_fe_10_min
0,CUS_0x100b,2024-06-01,2024-06-01,0,Unknown,0,0.000000,0.000000,0.0,0,...,1,152.0,152,0.0,180.0,180,62.0,62,233.0,233
1,CUS_0x1011,2024-06-01,2024-06-01,0,Unknown,0,0.000000,0.000000,0.0,0,...,1,68.0,68,0.0,237.0,237,223.0,223,-47.0,-47
2,CUS_0x1013,2024-06-01,2024-06-01,0,Unknown,0,0.000000,0.000000,0.0,0,...,1,92.0,92,0.0,94.0,94,122.0,122,39.0,39
3,CUS_0x1015,2024-06-01,2024-06-01,0,Unknown,0,0.000000,0.000000,0.0,0,...,1,32.0,32,0.0,219.0,219,71.0,71,233.0,233
4,CUS_0x1018,2024-06-01,2024-06-01,0,Unknown,0,0.000000,0.000000,0.0,0,...,1,220.0,220,0.0,105.0,105,214.0,214,106.0,106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5413,CUS_0xfe4,2024-06-01,2024-06-01,0,Unknown,0,0.000000,0.000000,0.0,0,...,1,203.0,203,0.0,-5.0,-5,168.0,168,246.0,246
5414,CUS_0xfea,2024-06-01,2024-06-01,0,Unknown,0,0.000000,0.000000,0.0,0,...,1,152.0,152,0.0,56.0,56,290.0,290,87.0,87
5415,CUS_0xff3,2024-06-01,2024-06-01,55,Scientist,13,1229.079956,81.195854,2.0,0,...,1,129.0,129,0.0,-40.0,-40,110.0,110,16.0,16
5416,CUS_0xffc,2024-06-01,2024-06-01,0,Unknown,0,0.000000,0.000000,0.0,0,...,1,71.0,71,0.0,-165.0,-165,174.0,174,162.0,162


## preprocess data for modeling

In [58]:
# prepare X_inference

# Define what columns are NOT features
non_feature_cols = [
    'Customer_ID', 'snapshot_date', 'label', 'loan_id', 
    'label_definition', 'label_snapshot_date', 'feature_snapshot_date', 'Occupation',
    'mob', 'dpd', 'dpd_mean', 'dpd_max', 'Loan_overdue_amt_sum', 'Loan_overdue_amt_mean',
    'Loan_overdue_amt_max', 'Loan_amt_sum', 'Loan_amt_mean', 'Loan_amt_std',
    'Loan_balance_sum', 'Loan_balance_mean', 'loan_count', 'Delay_from_due_date',
    'clickstream_total_events',
    'Age'
]

# All other columns are features 
feature_cols = [column for column in feature_sdf.columns if column not in non_feature_cols]

X_inference = feature_sdf[feature_cols]

scaler = model_artefact["preprocessing_transformers"]["stdscaler"]
train_features = model_artefact["feature_names"]

# Convert to pandas DataFrame if it's Spark
X_inference = X_inference.toPandas() if hasattr(X_inference, "toPandas") else X_inference

# Ensure only numeric data
X_inference = X_inference.apply(pd.to_numeric, errors="raise")

# Apply scaler (keeps column names)
transformer_stdscaler = model_artefact["preprocessing_transformers"]["stdscaler"]
X_inference_scaled = scaler.transform(X_inference)

# Convert back to DataFrame if needed
X_inference = pd.DataFrame(X_inference_scaled, columns=X_inference.columns)

print('X_inference rows:', X_inference.shape[0])
X_inference

X_inference rows: 5418


,Outstanding_Debt,Amount_invested_monthly,Interest_Rate,Num_Bank_Accounts,Num_Credit_Card,Loan_Type_Home_Loan,Loan_Type_Personal_Loan,Loan_Type_Student_Loan,Loan_Type_Auto_Loan,Loan_Type_Business_Loan,...,Loan_tenure_max,clickstream_fe_5_mean,clickstream_fe_5_sum,clickstream_fe_5_std,clickstream_fe_9_mean,clickstream_fe_9_min,clickstream_fe_4_mean,clickstream_fe_4_min,clickstream_fe_10_mean,clickstream_fe_10_min
0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.439982,0.439982,0.0,0.653974,0.653974,-0.427901,-0.427901,1.143284,1.143284
1,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.387444,-0.387444,0.0,1.229509,1.229509,1.181404,1.181404,-1.665745,-1.665745
2,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.151036,-0.151036,0.0,-0.214378,-0.214378,0.171840,0.171840,-0.802972,-0.802972
3,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.742055,-0.742055,0.0,1.047761,1.047761,-0.337940,-0.337940,1.143284,1.143284
4,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.109803,1.109803,0.0,-0.103309,-0.103309,1.091443,1.091443,-0.130812,-0.130812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5413,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.942348,0.942348,0.0,-1.213992,-1.213992,0.631641,0.631641,1.273703,1.273703
5414,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.439982,0.439982,0.0,-0.598068,-0.598068,1.851115,1.851115,-0.321424,-0.321424
5415,1229.079956,81.195854,2.0,0.0,6.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.213425,0.213425,0.0,-1.567391,-1.567391,0.051892,0.051892,-1.033714,-1.033714
5416,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,-0.357893,-0.357893,0.0,-2.829530,-2.829530,0.691616,0.691616,0.430994,0.430994


## model prediction inference

In [59]:
# load model
model = model_artefact["model"]

# predict model
y_inference = model.predict_proba(X_inference)[:, 1]

# prepare output
y_inference_pdf = feature_pdf[["Customer_ID","snapshot_date"]].copy()
y_inference_pdf["model_name"] = config["model_name"]
y_inference_pdf["model_predictions"] = y_inference
y_inference_pdf

,Customer_ID,snapshot_date,model_name,model_predictions
0,CUS_0x100b,2024-06-01,credit_model_2024_06_01.pkl,0.182158
1,CUS_0x1011,2024-06-01,credit_model_2024_06_01.pkl,0.361734
2,CUS_0x1013,2024-06-01,credit_model_2024_06_01.pkl,0.345503
3,CUS_0x1015,2024-06-01,credit_model_2024_06_01.pkl,0.167240
4,CUS_0x1018,2024-06-01,credit_model_2024_06_01.pkl,0.379753
...,...,...,...,...
5413,CUS_0xfe4,2024-06-01,credit_model_2024_06_01.pkl,0.309600
5414,CUS_0xfea,2024-06-01,credit_model_2024_06_01.pkl,0.426680
5415,CUS_0xff3,2024-06-01,credit_model_2024_06_01.pkl,0.389518
5416,CUS_0xffc,2024-06-01,credit_model_2024_06_01.pkl,0.338888


## save model inference to datamart gold table

In [60]:
# create bronze datalake
gold_directory = f'/app/datamart/gold/model_predictions/{config["model_name"][:-4]}/'
print(gold_directory)

if not os.path.exists(gold_directory):
    os.makedirs(gold_directory)

# save gold table - IRL connect to database to write
partition_name = config["model_name"][:-4] + "_predictions_" + snapshot_date_str.replace('-','_') + '.parquet'
filepath = gold_directory + partition_name
spark.createDataFrame(y_inference_pdf).write.mode("overwrite").parquet(filepath)
# df.toPandas().to_parquet(filepath,
#           compression='gzip')
print('saved to:', filepath)

/app/datamart/gold/model_predictions/credit_model_2024_06_01/


saved to: /app/datamart/gold/model_predictions/credit_model_2024_06_01/credit_model_2024_06_01_predictions_2024_06_01.parquet


## backfill

In [ ]:
# set up config
snapshot_date_str = "2023-01-01"

start_date_str = "2023-01-01"
end_date_str = "2024-12-01"

In [ ]:
# generate list of dates to process
def generate_first_of_month_dates(start_date_str, end_date_str):
    # Convert the date strings to datetime objects
    start_date = datetime.strptime(start_date_str, "%Y-%m-%d")
    end_date = datetime.strptime(end_date_str, "%Y-%m-%d")
    
    # List to store the first of month dates
    first_of_month_dates = []

    # Start from the first of the month of the start_date
    current_date = datetime(start_date.year, start_date.month, 1)

    while current_date <= end_date:
        # Append the date in yyyy-mm-dd format
        first_of_month_dates.append(current_date.strftime("%Y-%m-%d"))
        
        # Move to the first of the next month
        if current_date.month == 12:
            current_date = datetime(current_date.year + 1, 1, 1)
        else:
            current_date = datetime(current_date.year, current_date.month + 1, 1)

    return first_of_month_dates

dates_str_lst = generate_first_of_month_dates(start_date_str, end_date_str)


In [ ]:
for snapshot_date in dates_str_lst:
    print(snapshot_date)
    model_inference.main(snapshot_date, model_name)

## Check datamart

In [61]:
# Initialize SparkSession
spark = pyspark.sql.SparkSession.builder \
    .appName("dev") \
    .master("local[*]") \
    .getOrCreate()

# Set log level to ERROR to hide warnings
spark.sparkContext.setLogLevel("ERROR")

In [62]:
folder_path = "/app/datamart/gold/model_predictions/credit_model_2024_12_01/"
files_list = [folder_path+os.path.basename(f) for f in glob.glob(os.path.join(folder_path, '*'))]
df = spark.read.option("header", "true").parquet(*files_list)
print("row_count:",df.count())

df.show()

row_count: 5531
+-----------+-------------+--------------------+-------------------+
|Customer_ID|snapshot_date|          model_name|  model_predictions|
+-----------+-------------+--------------------+-------------------+
| CUS_0xbdc7|   2024-12-01|credit_model_2024...|   0.28524249792099|
| CUS_0xbdca|   2024-12-01|credit_model_2024...|0.19894181191921234|
| CUS_0xbde6|   2024-12-01|credit_model_2024...| 0.3585490584373474|
| CUS_0xbded|   2024-12-01|credit_model_2024...| 0.3087739944458008|
| CUS_0xbdee|   2024-12-01|credit_model_2024...| 0.3087739944458008|
| CUS_0xbdf7|   2024-12-01|credit_model_2024...| 0.2982424199581146|
| CUS_0xbe00|   2024-12-01|credit_model_2024...| 0.3087739944458008|
| CUS_0xbe03|   2024-12-01|credit_model_2024...| 0.3087739944458008|
| CUS_0xbe0a|   2024-12-01|credit_model_2024...| 0.3087739944458008|
|  CUS_0xbe1|   2024-12-01|credit_model_2024...|0.27329692244529724|
| CUS_0xbe1d|   2024-12-01|credit_model_2024...| 0.3087739944458008|
| CUS_0xbe26|   20